There will be a lot decisions made regarding features based on the analysis performed earlier. It can accessed in analysis.ipynb file.

In [10]:
import pandas as pd
import numpy as np
import datetime 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

from sklearn.externals import joblib
import pickle

In [2]:
creditCardData = pd.read_csv("Data/creditcard.csv")

In [3]:
# Feature engineering

# As the time provided is in seconds we can use it as seconds since epoch as we won't care about years
def convert_totime(seconds):
    return datetime.datetime.fromtimestamp(seconds);

creditCardData['datetime'] = creditCardData.Time.apply(convert_totime)
creditCardData['hour of the day'] = creditCardData.datetime + pd.Timedelta("7 hour")
creditCardData['hour of the day'] = creditCardData['hour of the day'].dt.hour
creditCardData['isNight'] = (creditCardData['hour of the day'] >= 1) & (creditCardData['hour of the day'] <= 7)

#dropping insignificant rows
creditCardData = creditCardData.drop(['V22','V23','V25','V26','V13','V15', 'datetime'], axis = 1)

In [4]:
creditCardData['isNight'] = creditCardData['isNight'].map({False:0, True:1})
train, test  = train_test_split(creditCardData, test_size=0.33, random_state=42)

In [12]:
model = RandomForestClassifier(n_estimators = 500, max_depth = 8, max_features = None)
train_y = train.Class
train_x = train.drop('Class', axis = 1)
model.fit(train_x, train_y)
test_y = test.Class
test_x = test.drop('Class', axis = 1)
preds = model.predict(test_x)
preds_prob = model.predict_proba(test_x)

In [19]:
print 'Accuracy:', accuracy_score(test_y, preds)
print 'AUC_ROC:', roc_auc_score(test_y, [x[1] for x in preds_prob])

Accuracy: 0.999595688765
AUC_ROC: 0.973519621349


The parameters n_estimators, max_depth and max_features have been tuned using Grid Search and Cross Validation